In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00


In [ ]:
#!pip install -U weaviate-client
!pip install "weaviate-client==3.*" # Version 4 of the client does not work

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os

import json
import re

In [ ]:
WCS_ENDPOINT = "https://my-sandbox1-1486fdzz.weaviate.network/" # Weaviate
WCS_API_KEY = "YOUR_WCS_API_KEY" # Weaviate
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'
COHERE_API_KEY = 'YOUR_COHERE_API_KEY'

VECTOR_DB_NAME = 'KAGGLE_DOCS_VDB'
BATCH_SIZE = 100

#SAMPLE_SIZE = 500

In [ ]:
base_path = '../input/gemma-comp-data'

In [ ]:
os.listdir('../input/gemma-comp-data/cleaned-txt-kaggle')

['publicapi.txt',
 'notebooks.txt',
 'tpu.txt',
 'organizations.txt',
 'datasets.txt',
 'models.txt',
 'efficientgou.txt',
 'competitionssetup.txt',
 'meetourteam.txt',
 'communityguidelines.txt',
 'competitions.txt',
 'privacypolicy.txt']

## Load the Kaggle data

In [ ]:
file_list = os.listdir('../input/gemma-comp-data/cleaned-txt-kaggle')

print('Num files:', len(file_list))

file_list

Num files: 12


['publicapi.txt',
 'notebooks.txt',
 'tpu.txt',
 'organizations.txt',
 'datasets.txt',
 'models.txt',
 'efficientgou.txt',
 'competitionssetup.txt',
 'meetourteam.txt',
 'communityguidelines.txt',
 'competitions.txt',
 'privacypolicy.txt']

In [ ]:
# Example: Load one txt file

fname = file_list[8]
file_path = base_path + '/cleaned-txt-kaggle/' + fname

with open(file_path, "r") as file:

    # read the file
    content = file.read()

    # split by the # symbol
    lines = content.split("###")

    # create the chunks
    chunk_list = [line.strip() for line in lines]

print(len(chunk_list))


# Create a dataframe

cols = ['text_chunk']
df = pd.DataFrame(chunk_list, columns=cols)

# Add the fname
df['fname'] = fname

print(df.shape)

df.head()

64
(64, 2)


,text_chunk,fname
0,Kaggle Team Members (Employees) and their prof...,meetourteam.txt
1,{Kaggle team member} Julia Elliott\nCOO\nJulia...,meetourteam.txt
2,{Kaggle team member} Jeff Moser\nChief Archite...,meetourteam.txt
3,{Kaggle team member} William Cukierski\nHead o...,meetourteam.txt
4,{Kaggle team member} Jerad Rose\nDeveloper\nJe...,meetourteam.txt


In [ ]:
# Load all txt files

for i, fname in enumerate(file_list):

    file_path = base_path + '/cleaned-txt-kaggle/' + fname

    with open(file_path, "r") as file:

        # read the file
        content = file.read()

        # split by the # symbol
        lines = content.split("###")

        # create the chunks
        chunk_list = [line.strip() for line in lines]

    # Create a dataframe

    cols = ['text_chunk']
    df = pd.DataFrame(chunk_list, columns=cols)

    # Add the fname
    df['fname'] = fname

    if i == 0:
        df_data = df.copy()
    else:
        df_data = pd.concat([df_data, df], axis=0)


# Reset the index
df_data = df_data.reset_index(drop=True)

print(df_data.shape)

df_data.head()

(160, 2)


,text_chunk,fname
0,{Kaggle Docs: Public API} Public API\nCreate D...,publicapi.txt
1,{Kaggle Docs: Public API} Getting Started: Ins...,publicapi.txt
2,{Kaggle Docs: Public API} Interacting with Com...,publicapi.txt
3,{Kaggle Docs: Public API} Interacting with Dat...,publicapi.txt
4,{Kaggle Docs: Public API} Interacting with Not...,publicapi.txt


In [ ]:
df_data['fname'].unique()

array(['publicapi.txt', 'notebooks.txt', 'tpu.txt', 'organizations.txt',
       'datasets.txt', 'models.txt', 'efficientgou.txt',
       'competitionssetup.txt', 'meetourteam.txt',
       'communityguidelines.txt', 'competitions.txt', 'privacypolicy.txt'],
      dtype=object)

## Clean the data

In [ ]:
# Replace newline characters ('\n') with a space
# Remove leading and trailing spaces

def clean_text(x):

    # Replace newline characters with a space
    new_text = x.replace("\n", " ")
    # Remove leading and trailing spaces
    new_text = new_text.strip()

    return new_text

df_data['prepared_text'] = df_data['text_chunk'].apply(clean_text)

df_data.head()

,text_chunk,fname,prepared_text
0,{Kaggle Docs: Public API} Public API\nCreate D...,publicapi.txt,{Kaggle Docs: Public API} Public API Create Da...
1,{Kaggle Docs: Public API} Getting Started: Ins...,publicapi.txt,{Kaggle Docs: Public API} Getting Started: Ins...
2,{Kaggle Docs: Public API} Interacting with Com...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Com...
3,{Kaggle Docs: Public API} Interacting with Dat...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Dat...
4,{Kaggle Docs: Public API} Interacting with Not...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Not...


## Get the total number of tokens

In [ ]:
# Ref: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

import tiktoken

# Function to calculate the number of tokens in a string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
# Example
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [ ]:
def get_num_tokens(x):

    num_tokens = num_tokens_from_string(x, "cl100k_base")

    return num_tokens

df_data['num_tokens'] = df_data['prepared_text'].apply(get_num_tokens)

print(df_data.shape)
print(df_data['num_tokens'].sum())

df_data.head()

(160, 4)
43873


,text_chunk,fname,prepared_text,num_tokens
0,{Kaggle Docs: Public API} Public API\nCreate D...,publicapi.txt,{Kaggle Docs: Public API} Public API Create Da...,40
1,{Kaggle Docs: Public API} Getting Started: Ins...,publicapi.txt,{Kaggle Docs: Public API} Getting Started: Ins...,397
2,{Kaggle Docs: Public API} Interacting with Com...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Com...,396
3,{Kaggle Docs: Public API} Interacting with Dat...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Dat...,766
4,{Kaggle Docs: Public API} Interacting with Not...,publicapi.txt,{Kaggle Docs: Public API} Interacting with Not...,747


## Estimate the cost for creating the embeddings

In [ ]:
# https://openai.com/pricing
# at 22-Feb-2024

# text-embedding-3-small	$0.00002 / 1K tokens # US Dollars
# text-embedding-3-large	$0.00013 / 1K tokens # US Dollars
# ada v2	$0.00010 / 1K tokens # US Dollars

In [ ]:
total_tokens = df_data['num_tokens'].sum()
cost_per_1000_tokens = 0.00002 #  text-embedding-3-small

total_cost = (total_tokens/1000) * cost_per_1000_tokens

print('Num papers:',len(df_data))
print('Total tokens:',total_tokens)
print('Total cost (USD):',total_cost) # US Dollars

Num papers: 160
Total tokens: 43873
Total cost (USD): 0.00087746


In [ ]:
#qqq

## Create a list of dicts

In [ ]:
# Example data format for upload to the vector database

data = [
   {
      "title": "Object0",
      "foo": 99,
      "quote_text": "The quick brown fox jumps over the lazy dog."
   },
   {
      "title": "Object1",
      "foo": 77,
      "quote_text": "A nimble red fox leaped over the sleeping hound."
   }
]


In [ ]:
data_list = []

df_final = df_data.copy()

for i in range(0, len(df_final)):

    data = {
        "prepared_text": df_final.loc[i, 'prepared_text']
    }

    data_list.append(data)

len(data_list)

160

In [ ]:
data_list[3]

{'prepared_text': '{Kaggle Docs: Public API} Interacting with Datasets  The Kaggle API and CLI tool provide easy ways to interact with Datasets on Kaggle. The commands available can make searching for and downloading Kaggle Datasets a seamless part of your data science workflow.  If you haven’t installed the Kaggle Python package needed to use the command line tool or generated an API token, check out the getting started steps first.  Some of the commands for interacting with Datasets via CLI include:  - kaggle datasets list -s [KEYWORD]: list datasets matching a search term - kaggle datasets download -d [DATASET]: download files associated with a dataset  If you are creating or updating a dataset on Kaggle, you can also use the API to make maintenance convenient or even programmatic. View all available commands on the official documentation on GitHub and keep up-to-date with the latest features and bug fixes in the changelog.  To explore additional CLI arguments, remember that you can

## Create the vector database

In [ ]:
# Connect

import weaviate
import json

client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

client.is_ready()

/opt/conda/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


True

In [ ]:
# https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-openai#api-settings-openai
# Configure the database

# resetting the schema. CAUTION: This will delete your collection
if client.schema.exists(VECTOR_DB_NAME):
     client.schema.delete_class(VECTOR_DB_NAME)


# Create a Table in the database.
# We call this table a "class".

class_obj = {
    "class": VECTOR_DB_NAME, # This is the name of the database table
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {
            # "model": "ada",
            # "modelVersion": "002",
          #"model": "text-embedding-3-large",
          #"dimensions": 3072,
          "model": "text-embedding-3-small",
          "dimensions": 1536,
            "type": "text",
        },
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

## Upload the data to the vector database

In [ ]:
BATCH_SIZE

100

In [ ]:
import requests
import json
from tqdm import tqdm

client.batch.configure(batch_size=BATCH_SIZE)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, data in enumerate(tqdm(data_list, total=len(data_list))):  # Batch import data
        #print(f"importing chunk: {i+1}")
        properties = {
            "prepared_text": data["prepared_text"]
        }
        batch.add_data_object(
            data_object=properties,
            class_name=VECTOR_DB_NAME
        )

100%|██████████| 160/160 [00:11<00:00, 14.27it/s]


In [ ]:
# Check number of objects in MyTable1

response = (
    client.query
    .aggregate(VECTOR_DB_NAME)
    .with_meta_count()
    .do()
)

print(response)

{'data': {'Aggregate': {'KAGGLE_DOCS_VDB': [{'meta': {'count': 160}}]}}}


## Example: Query the database

Here we will run a hybrid search.

In [ ]:
# Connect to the database

import weaviate
import json

wcs_client = weaviate.Client(
    url = WCS_ENDPOINT,  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=WCS_API_KEY),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Cohere-Api-Key": COHERE_API_KEY
    }
)

# check that the client is ready
wcs_client.is_ready()

True

In [ ]:
#query_text = "General purpose computers"
query_text = "Who is the CEO of Kaggle?"

top_k = 10

response = (
    wcs_client.query
    .get(VECTOR_DB_NAME, ["prepared_text"])
    .with_near_text({"concepts": [query_text]})
    .with_limit(top_k)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"])
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "KAGGLE_DOCS_VDB": [
                {
                    "_additional": {
                        "distance": 0.30767936,
                        "id": "3330519f-0bf2-4e6a-a6bd-c9a7128a400a"
                    },
                    "prepared_text": "Kaggle Team Members (Employees) and their profiles Link to section in Kaggle website: https://www.kaggle.com/about/team   {Kaggle team member} D. Sculley CEO D. is the CEO at Kaggle. Prior to coming to Kaggle, he was a director at Google Brain, leading research teams working on robust, responsible, reliable and efficient ML and AI. In his career in ML, he has worked on nearly every aspect of machine learning, and has led both product and research teams including those on some of the most challenging business problems. Some of his well known work involves ML Technical Debt, ML Education, ML Robustness, production-critical ML, and ML for scientific applications such as protein design."
        

In [ ]:
# Run a hybrid search

response = (
    wcs_client.query
    .get(VECTOR_DB_NAME, ["prepared_text"])
    .with_hybrid(query=query_text, alpha=0.5)
    .with_limit(top_k)
    #.with_additional(["distance", "vector, id"]) # Also return the vector, the distance and the id
    .with_additional(["distance", "id"]) # This id is the Weaviate database id
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "KAGGLE_DOCS_VDB": [
                {
                    "_additional": {
                        "distance": null,
                        "id": "3330519f-0bf2-4e6a-a6bd-c9a7128a400a"
                    },
                    "prepared_text": "Kaggle Team Members (Employees) and their profiles Link to section in Kaggle website: https://www.kaggle.com/about/team   {Kaggle team member} D. Sculley CEO D. is the CEO at Kaggle. Prior to coming to Kaggle, he was a director at Google Brain, leading research teams working on robust, responsible, reliable and efficient ML and AI. In his career in ML, he has worked on nearly every aspect of machine learning, and has led both product and research teams including those on some of the most challenging business problems. Some of his well known work involves ML Technical Debt, ML Education, ML Robustness, production-critical ML, and ML for scientific applications such as protein design."
              

In [ ]:
# Include reranking after the hybrid search

response = (
    wcs_client.query
    .get(VECTOR_DB_NAME, ["prepared_text"])
    .with_hybrid(query=query_text, alpha=0.5)
    .with_additional(f"rerank(property: \"prepared_text\" query: \"{query_text}\") {{ score }}")
    .with_limit(top_k)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "KAGGLE_DOCS_VDB": [
                {
                    "_additional": {
                        "rerank": [
                            {
                                "score": 0.9999998
                            }
                        ]
                    },
                    "prepared_text": "Kaggle Team Members (Employees) and their profiles Link to section in Kaggle website: https://www.kaggle.com/about/team   {Kaggle team member} D. Sculley CEO D. is the CEO at Kaggle. Prior to coming to Kaggle, he was a director at Google Brain, leading research teams working on robust, responsible, reliable and efficient ML and AI. In his career in ML, he has worked on nearly every aspect of machine learning, and has led both product and research teams including those on some of the most challenging business problems. Some of his well known work involves ML Technical Debt, ML Education, ML Robustness, production-critical ML, and ML for s

## Use OpenAI to create a natural language output

In [ ]:
# Get the chunks for the context
response_list = response['data']['Get'][VECTOR_DB_NAME]

len(response_list)

10

In [ ]:
text = response_list[0]['prepared_text']

text

'Kaggle Team Members (Employees) and their profiles Link to section in Kaggle website: https://www.kaggle.com/about/team   {Kaggle team member} D. Sculley CEO D. is the CEO at Kaggle. Prior to coming to Kaggle, he was a director at Google Brain, leading research teams working on robust, responsible, reliable and efficient ML and AI. In his career in ML, he has worked on nearly every aspect of machine learning, and has led both product and research teams including those on some of the most challenging business problems. Some of his well known work involves ML Technical Debt, ML Education, ML Robustness, production-critical ML, and ML for scientific applications such as protein design.'

In [ ]:
query_text

'Who is the CEO of Kaggle?'

In [ ]:
# Extract just the text

pred_text_list = []

for i in range(0, len(response_list)):

    pred_text = response_list[i]['prepared_text']

    pred_text_list.append(pred_text)

len(pred_text_list)

10

In [ ]:
# Choose the first three text chunks
context = pred_text_list[0:3]

# Prepare the prompt
prompt = f"""
Text containing information:
{context}
Question: {query_text}

Extract the answer to the question from the text provided.
If the text doesn't contain the answer,
reply that the answer is not available."""

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)


print(completion.choices[0].message.content)

The CEO of Kaggle is D. Sculley.
